In [1]:
# %matplotlib inline
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

# IMPORT

In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import xarray as xr


from pycns import CnsStream, CnsReader, get_viewer
import physio

from physio_rea_analyse import *
from custom_view import *

from configuration import *
from tools import *
from params import *

from icp_jobs import *

# READER

In [4]:
subs

['Patient_2024_May_16__9_33_08_427295',
 'Patient_2024_May_13__9_36_06_745131',
 'Patient_2024_May_8__9_51_19_328502',
 'Patient_2024_May_25__16_09_03_881877',
 'Patient_2024_Sep_10__11_32_35_613533',
 'Patient_2024_May_25__16_48_44_716049',
 'Patient_2024_Jun_24__12_33_38_160174',
 'PatientData_1723890119868125']

In [5]:
# patient_id = 'Patient_2024_May_16__9_33_08_427295'
# patient_id = 'Patient_2024_May_25__16_48_44_716049'
# patient_id = 'Patient_2024_May_25__16_09_03_881877'
# patient_id = 'Patient_2024_Jun_24__12_33_38_160174'
# patient_id = 'Patient_2024_Jun_24__12_33_38_160174'
patient_id = 'Patient_2024_May_8__9_51_19_328502'

raw_folder = data_path / patient_id
cns_reader = CnsReader(raw_folder, with_events=True, event_time_zone='Europe/Paris')

In [6]:
meta = get_metadata(patient_id)
meta

Unnamed: 0                        NaN
folder                     projet_DVE
Unnamed: 3                        NaN
debut             2024-05-08 10:00:00
fin               2024-05-08 17:35:00
PIC                               ICP
PIV                                P1
Numero                              3
Sexe                                H
Age                              57.0
Date_admission    2024-04-25 00:00:00
Date_sortie       2024-06-03 00:00:00
Duree_sejour                     39.0
Indication                        1.0
Date_pose         2024-04-26 00:00:00
Date_ablation     2024-05-14 00:00:00
Duree_DVE                        18.0
Kakarla                           1.0
EIDVE                             0.0
DVI                               1.0
Date_DVI          2024-05-24 00:00:00
Delais_DVI                       10.0
VVTDM0                            NaN
VVTDM1                            NaN
VVTDM2                            NaN
VVTDM3                            NaN
VSTDM1      

In [7]:
pd.DataFrame(cns_reader.events)

,name,start_time,duration,description
0,1:declampage,2024-05-08 15:37:37.230,0.0,


# PRECOMPUTE

In [8]:
# force_compute = False

# if force_compute:
#     icp_stream = cns_reader.streams['ICP']
#     srate_icp = icp_stream.sample_rate
#     raw_icp, dates = icp_stream.get_data(with_times=True, apply_gain=True)
#     icp_features = compute_icp(raw_icp, srate_icp, date_vector = dates)
# else:
#     icp_features = detect_icp_job.get(patient_id).to_dataframe()
# print(icp_features.shape)

# VIEWER RAW

In [9]:
cns_reader.streams.keys()

dict_keys(['SpO2', 'RR', 'ICP', 'ABP_Syst', 'HR', 'ECG_II', 'P1_Dias', 'ABP', 'P1_Mean', 'ABP_Mean', 'P1', 'P1_Syst', 'ABP_Dias', 'PLETH', 'ICP_Mean', 'RESP'])

In [10]:
stream_names = list(cns_reader.streams.keys())
stream_with_P = [name for name in stream_names if 'P' in name and len(name) < 4]
stream_with_P = [name for name in stream_with_P if not name in ['CPP','ABP']]
stream_with_P = stream_with_P + [f'{name}_Mean' for name in stream_with_P]
stream_with_P

['ICP', 'P1', 'ICP_Mean', 'P1_Mean']

In [47]:
get_date_windows_gmt(patient_id)

{'debut': numpy.datetime64('2024-06-24T18:00:00.000000000'),
 'fin': numpy.datetime64('2024-06-25T06:43:00.000000000')}

In [11]:
patient_id

'Patient_2024_May_8__9_51_19_328502'

In [12]:
w = get_viewer(cns_reader, stream_names=stream_with_P, with_events = False)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# VIEWER EXTERNALS

In [25]:
icp_chan_name[patient_id]

'ICP'

In [26]:
icp_features = detect_icp_job.get(patient_id).to_dataframe()
stream_name = icp_chan_name[patient_id]
psi_da = psi_job.get(patient_id)['psi']

ext_plots = {
            'detections':ICP_Detections(cns_reader.streams[stream_name], icp_features),
            'psi':PSI(psi_da)
}

w = get_viewer(cns_reader, stream_names=[stream_name], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…